
---

# **Chapter 13: Riverpod (Next-Gen State Management)**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Understand the differences between Riverpod and Provider
- Set up Riverpod using `ProviderScope` and various provider types
- Use `StateProvider` for simple state and `StateNotifier` for complex state
- Implement `StateNotifierProvider` for immutable state management
- Apply modifiers like `autoDispose` and `family` for resource management
- Handle asynchronous data with `AsyncValue`
- Use `FutureProvider` and `StreamProvider` in Riverpod
- Test Riverpod providers without BuildContext
- Migrate from Provider to Riverpod architecture

---

## **Prerequisites**

- Completed Chapter 12: Provider Pattern
- Understanding of ChangeNotifier and state management concepts
- Knowledge of immutable state patterns
- Familiarity with Flutter widget lifecycle
- Basic understanding of code generation (optional but helpful)

---

## **13.1 Introduction to Riverpod**

Riverpod is a reactive caching and data-binding framework created by Remi Rousselet (the creator of Provider). While Provider is built on top of InheritedWidget, Riverpod is a complete rewrite that solves many of Provider's limitations.

### **Why Riverpod over Provider?**

```dart
// Provider limitations that Riverpod solves:

// 1. Provider requires BuildContext to read
// In Provider:
// final value = context.read<MyProvider>(); // Needs context!

// 2. Provider is not compile-safe
// If you forget to provide the value, you get a runtime error
// In Provider:
// context.read<MissingProvider>(); // Runtime error!

// 3. Provider doesn't auto-dispose by default
// You must manually manage disposal or use complicated workarounds

// 4. Provider makes testing difficult
// You need to wrap widgets in Provider widgets to test them
```

**Explanation:**

- **No BuildContext required**: Riverpod providers are global constants that can be accessed from anywhere (even outside the widget tree), making them much more flexible.
- **Compile-time safety**: Riverpod catches errors at compile time rather than runtime. If a provider doesn't exist, your code won't compile.
- **Auto-dispose**: Riverpod automatically disposes providers when they're no longer used, preventing memory leaks by default.
- **Testability**: You can test Riverpod providers in isolation without building widget trees or mocking BuildContext.
- **Flexibility**: Riverpod works independently of Flutter (can be used in Dart-only projects), though it integrates seamlessly with Flutter.

### **Setting Up Riverpod**

```dart
// pubspec.yaml dependencies:
// dependencies:
//   flutter_riverpod: ^2.4.0  # For Flutter integration
//   riverpod_annotation: ^2.0.0  # For code generation (optional)

// main.dart - The entry point changes significantly
import 'package:flutter/material.dart';
import 'package:flutter_riverpod/flutter_riverpod.dart';

void main() {
  runApp(
    // ProviderScope is the root widget that stores the state of all providers
    // This is similar to how ChangeNotifierProvider works, but for all Riverpod providers
    ProviderScope(
      child: MyApp(),
    ),
  );
}

class MyApp extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'Riverpod Demo',
      home: HomeScreen(),
    );
  }
}

// A simple provider that holds a string value
// Providers are declared as global final variables
// This is a Provider<String>, the most basic type
final greetingProvider = Provider<String>((ref) {
  // The ref parameter allows interacting with other providers
  return 'Hello, Riverpod!';
});

// Reading a provider in a widget
class HomeScreen extends ConsumerWidget {
  // ConsumerWidget is like StatelessWidget but with a WidgetRef parameter
  @override
  Widget build(BuildContext context, WidgetRef ref) {
    // ref.watch listens to the provider and rebuilds when it changes
    final greeting = ref.watch(greetingProvider);
    
    return Scaffold(
      appBar: AppBar(title: Text('Riverpod Basics')),
      body: Center(
        child: Text(
          greeting,
          style: Theme.of(context).textTheme.headline4,
        ),
      ),
    );
  }
}
```

**Explanation:**

- **`ProviderScope`**: The root widget that must wrap your app. It creates the "container" that holds all provider states. Without this, you cannot use any Riverpod providers.
- **Global providers**: Unlike Provider where you define providers in the widget tree, Riverpod providers are defined as global variables. This makes them accessible from anywhere.
- **`Provider<T>`**: The most basic provider type. It computes a value once and caches it. It doesn't have state that changes over time (for that, use StateProvider).
- **The `ref` parameter**: Short for "reference", this object allows you to interact with other providers (read, watch, listen).
- **`ref.watch(provider)`**: Subscribes to a provider. The widget rebuilds whenever the provider's value changes.
- **`ConsumerWidget`**: A convenience widget that provides the `WidgetRef ref` parameter in the build method, eliminating the need for `Consumer` builders.
- **Type safety**: `greetingProvider` is typed as `Provider<String>`, so `ref.watch(greetingProvider)` returns a `String`, not `dynamic`.

### **Provider Types Overview**

```dart
import 'package:flutter_riverpod/flutter_riverpod.dart';

// 1. Provider - Immutable, read-only values
final configProvider = Provider<AppConfig>((ref) {
  return AppConfig(apiUrl: 'https://api.example.com', version: '1.0');
});

// 2. StateProvider - Simple mutable state (primitives, simple objects)
final counterProvider = StateProvider<int>((ref) => 0);

// 3. StateNotifierProvider - Complex mutable state with logic
final userProvider = StateNotifierProvider<UserNotifier, UserState>((ref) {
  return UserNotifier();
});

// 4. FutureProvider - Async values (API calls, database queries)
final postsProvider = FutureProvider<List<Post>>((ref) async {
  final response = await http.get(Uri.parse('https://api.example.com/posts'));
  return parsePosts(response.body);
});

// 5. StreamProvider - Real-time data streams
final messagesProvider = StreamProvider<List<Message>>((ref) {
  return FirebaseFirestore.instance
      .collection('messages')
      .snapshots()
      .map((snapshot) => snapshot.docs.map((doc) => Message.fromDoc(doc)).toList());
});

// 6. ChangeNotifierProvider - For migrating from Provider package (not recommended for new code)
final legacyProvider = ChangeNotifierProvider<LegacyController>((ref) {
  return LegacyController();
});
```

**Explanation:**

- **`Provider`**: For immutable values that don't change (configuration, services, repositories). Computed once and cached.
- **`StateProvider`**: For simple state that can change (primitives like int, String, bool, or simple objects). Good for UI state like selected tab, counter, form input.
- **`StateNotifierProvider`**: For complex business logic with immutable state. The recommended way for application state. Combines well with Freezed or built_value for immutability.
- **`FutureProvider`**: Handles asynchronous operations automatically. Returns an `AsyncValue<T>` that represents loading, error, or data states.
- **`StreamProvider`**: Listens to Dart Streams (Firebase, WebSockets, etc.). Also returns `AsyncValue<T>`.
- **`ChangeNotifierProvider`**: Exists for migration from Provider package. Not recommended for new Riverpod code because StateNotifier is more testable and has better auto-dispose behavior.

---

## **13.2 StateProvider for Simple State**

`StateProvider` is the simplest way to manage mutable state in Riverpod. It's ideal for primitives (int, String, bool) and simple objects that can be replaced entirely.

### **Basic StateProvider Usage**

```dart
import 'package:flutter/material.dart';
import 'package:flutter_riverpod/flutter_riverpod.dart';

// Declare a StateProvider globally
// StateProvider<int> holds an integer state that starts at 0
final counterProvider = StateProvider<int>((ref) {
  // The callback returns the initial value
  return 0;
});

// Another example: theme toggle
final isDarkModeProvider = StateProvider<bool>((ref) => false);

// String state for search query
final searchQueryProvider = StateProvider<String>((ref) => '');

class CounterScreen extends ConsumerWidget {
  @override
  Widget build(BuildContext context, WidgetRef ref) {
    // Watch the provider to rebuild when state changes
    final count = ref.watch(counterProvider);
    
    return Scaffold(
      appBar: AppBar(
        title: Text('StateProvider Demo'),
        // Using Consumer widget for specific parts that need to rebuild
        actions: [
          Consumer(
            builder: (context, ref, child) {
              // This icon button only rebuilds when isDarkMode changes
              final isDark = ref.watch(isDarkModeProvider);
              return IconButton(
                icon: Icon(isDark ? Icons.light_mode : Icons.dark_mode),
                onPressed: () {
                  // Toggle the state
                  // ref.read gets the StateController, .state gets/sets the value
                  ref.read(isDarkModeProvider.notifier).state = !isDark;
                },
              );
            },
          ),
        ],
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            Text(
              'Count:',
              style: Theme.of(context).textTheme.headline6,
            ),
            Text(
              '$count',
              style: Theme.of(context).textTheme.headline2,
            ),
            SizedBox(height: 20),
            
            // Search field example
            Padding(
              padding: EdgeInsets.all(16),
              child: TextField(
                decoration: InputDecoration(
                  hintText: 'Search...',
                  prefixIcon: Icon(Icons.search),
                  border: OutlineInputBorder(),
                ),
                onChanged: (value) {
                  // Update the search query state
                  ref.read(searchQueryProvider.notifier).state = value;
                },
              ),
            ),
            
            // Display current search query
            Consumer(
              builder: (context, ref, child) {
                final query = ref.watch(searchQueryProvider);
                return Text('Searching for: $query');
              },
            ),
          ],
        ),
      ),
      floatingActionButton: Column(
        mainAxisAlignment: MainAxisAlignment.end,
        children: [
          FloatingActionButton(
            heroTag: 'increment',
            onPressed: () {
              // Increment the counter
              // .notifier gives us the StateController
              // .state is the current value
              ref.read(counterProvider.notifier).state++;
              
              // Alternative syntax (explicit):
              // ref.read(counterProvider.notifier).state = 
              //   ref.read(counterProvider) + 1;
            },
            child: Icon(Icons.add),
          ),
          SizedBox(height: 10),
          FloatingActionButton(
            heroTag: 'decrement',
            onPressed: () {
              ref.read(counterProvider.notifier).state--;
            },
            child: Icon(Icons.remove),
          ),
          SizedBox(height: 10),
          FloatingActionButton(
            heroTag: 'reset',
            onPressed: () {
              // Reset to initial value
              ref.read(counterProvider.notifier).state = 0;
            },
            child: Icon(Icons.refresh),
          ),
        ],
      ),
    );
  }
}
```

**Explanation:**

- **`StateProvider<T>`**: A provider that exposes a value that can be modified from outside. Unlike regular `Provider`, it has a `.notifier` property.
- **Initial value**: The callback returns the initial state (0 for counter, false for dark mode, empty string for search).
- **`ref.watch(counterProvider)`**: Returns the current state value (int). The widget rebuilds when this value changes.
- **`ref.read(counterProvider.notifier)`**: Returns a `StateController<T>` which has a `.state` property that can be read or written.
- **Updating state**: Assign a new value to `.state` to update the provider. Riverpod automatically notifies listeners.
- **`ref.read` vs `ref.watch`**:
  - Use `ref.watch` in build methods to subscribe to changes
  - Use `ref.read` in callbacks (onPressed, onChanged) to get the current value or controller without subscribing
- **`.notifier`**: Accessing the controller allows you to modify the state. Without `.notifier`, you only get the value.
- **Multiple providers**: You can declare and use as many StateProviders as needed. Each is independent.

### **StateProvider with Complex Objects**

```dart
// StateProvider can hold simple objects, but they should be immutable
// For complex mutable objects, use StateNotifierProvider instead

// A simple data class (using built-in Dart classes or Freezed)
class FilterState {
  final String category;
  final double minPrice;
  final double maxPrice;
  final bool onlyInStock;
  
  const FilterState({
    this.category = 'all',
    this.minPrice = 0,
    this.maxPrice = double.infinity,
    this.onlyInStock = false,
  });
  
  // Helper method to create a copy with modified fields
  FilterState copyWith({
    String? category,
    double? minPrice,
    double? maxPrice,
    bool? onlyInStock,
  }) {
    return FilterState(
      category: category ?? this.category,
      minPrice: minPrice ?? this.minPrice,
      maxPrice: maxPrice ?? this.maxPrice,
      onlyInStock: onlyInStock ?? this.onlyInStock,
    );
  }
}

// StateProvider holding the filter state
final filterProvider = StateProvider<FilterState>((ref) {
  return const FilterState(); // Initial state with defaults
});

class ProductFilterScreen extends ConsumerWidget {
  @override
  Widget build(BuildContext context, WidgetRef ref) {
    final filters = ref.watch(filterProvider);
    
    return Scaffold(
      appBar: AppBar(title: Text('Filters')),
      body: ListView(
        children: [
          // Category filter
          ListTile(
            title: Text('Category'),
            subtitle: Text(filters.category),
            trailing: DropdownButton<String>(
              value: filters.category,
              items: ['all', 'electronics', 'clothing', 'food']
                  .map((cat) => DropdownMenuItem(
                        value: cat,
                        child: Text(cat),
                      ))
                  .toList(),
              onChanged: (value) {
                if (value != null) {
                  // Update specific field while keeping others
                  ref.read(filterProvider.notifier).state = 
                      filters.copyWith(category: value);
                }
              },
            ),
          ),
          
          // Price range
          ListTile(
            title: Text('Max Price: \$${filters.maxPrice.toInt()}'),
            subtitle: Slider(
              value: filters.maxPrice == double.infinity ? 1000 : filters.maxPrice,
              min: 0,
              max: 1000,
              divisions: 20,
              label: '\$${filters.maxPrice.toInt()}',
              onChanged: (value) {
                ref.read(filterProvider.notifier).state = 
                    filters.copyWith(maxPrice: value);
              },
            ),
          ),
          
          // In stock toggle
          SwitchListTile(
            title: Text('Only In Stock'),
            value: filters.onlyInStock,
            onChanged: (value) {
              ref.read(filterProvider.notifier).state = 
                  filters.copyWith(onlyInStock: value);
            },
          ),
          
          // Reset button
          ElevatedButton(
            onPressed: () {
              // Reset to initial state
              ref.read(filterProvider.notifier).state = const FilterState();
            },
            child: Text('Reset Filters'),
          ),
        ],
      ),
    );
  }
}
```

**Explanation:**

- **Immutable state**: Even with StateProvider, prefer immutable objects. When updating, create a new instance rather than modifying the existing one.
- **`copyWith` pattern**: A standard Dart pattern for immutable objects. It creates a new instance with some fields changed and others copied from the original.
- **State replacement**: `ref.read(provider.notifier).state = newState` completely replaces the old state. This is fine for simple objects but can be verbose for complex nested updates.
- **When to use StateNotifier**: If you find yourself writing complex logic in the widget (validation, calculations, multiple related updates), move to StateNotifierProvider instead.

---

## **13.3 StateNotifier and StateNotifierProvider**

For complex business logic, `StateNotifier` combined with `StateNotifierProvider` is the recommended approach. It enforces immutability and separates business logic from UI.

### **Creating a StateNotifier**

```dart
import 'package:flutter_riverpod/flutter_riverpod.dart';

// The state class - should be immutable
// Using built_value or Freezed is recommended for production
class TodoState {
  final List<Todo> todos;
  final bool isLoading;
  final String? errorMessage;
  
  const TodoState({
    this.todos = const [],
    this.isLoading = false,
    this.errorMessage,
  });
  
  // Computed properties
  List<Todo> get completedTodos => 
      todos.where((t) => t.isCompleted).toList();
  
  List<Todo> get pendingTodos => 
      todos.where((t) => !t.isCompleted).toList();
  
  int get todoCount => todos.length;
  int get completedCount => completedTodos.length;
  
  // Copy method for immutability
  TodoState copyWith({
    List<Todo>? todos,
    bool? isLoading,
    String? errorMessage,
  }) {
    return TodoState(
      todos: todos ?? this.todos,
      isLoading: isLoading ?? this.isLoading,
      errorMessage: errorMessage, // Can be set to null explicitly
    );
  }
}

class Todo {
  final String id;
  final String title;
  final bool isCompleted;
  final DateTime createdAt;
  
  const Todo({
    required this.id,
    required this.title,
    this.isCompleted = false,
    required this.createdAt,
  });
  
  Todo copyWith({
    String? id,
    String? title,
    bool? isCompleted,
    DateTime? createdAt,
  }) {
    return Todo(
      id: id ?? this.id,
      title: title ?? this.title,
      isCompleted: isCompleted ?? this.isCompleted,
      createdAt: createdAt ?? this.createdAt,
    );
  }
}

// StateNotifier that manages TodoState
class TodoNotifier extends StateNotifier<TodoState> {
  // Constructor initializes with initial state
  TodoNotifier() : super(const TodoState());
  
  // Business logic methods
  
  Future<void> addTodo(String title) async {
    // Set loading state
    state = state.copyWith(isLoading: true, errorMessage: null);
    
    try {
      // Simulate API call
      await Future.delayed(Duration(milliseconds: 500));
      
      if (title.isEmpty) {
        throw Exception('Title cannot be empty');
      }
      
      final newTodo = Todo(
        id: DateTime.now().toString(),
        title: title,
        createdAt: DateTime.now(),
      );
      
      // Update state with new todo
      state = state.copyWith(
        todos: [...state.todos, newTodo], // Spread operator creates new list
        isLoading: false,
      );
    } catch (e) {
      state = state.copyWith(
        isLoading: false,
        errorMessage: e.toString(),
      );
    }
  }
  
  void toggleTodo(String id) {
    // Find the todo and create a new list with updated todo
    final updatedTodos = state.todos.map((todo) {
      if (todo.id == id) {
        return todo.copyWith(isCompleted: !todo.isCompleted);
      }
      return todo;
    }).toList();
    
    state = state.copyWith(todos: updatedTodos);
  }
  
  void removeTodo(String id) {
    state = state.copyWith(
      todos: state.todos.where((todo) => todo.id != id).toList(),
    );
  }
  
  Future<void> loadTodos() async {
    state = state.copyWith(isLoading: true);
    
    try {
      await Future.delayed(Duration(seconds: 1));
      // Simulate fetching from API
      final mockTodos = [
        Todo(id: '1', title: 'Learn Riverpod', createdAt: DateTime.now()),
        Todo(id: '2', title: 'Build an app', createdAt: DateTime.now()),
      ];
      
      state = state.copyWith(
        todos: mockTodos,
        isLoading: false,
      );
    } catch (e) {
      state = state.copyWith(
        isLoading: false,
        errorMessage: 'Failed to load todos',
      );
    }
  }
  
  void clearError() {
    if (state.errorMessage != null) {
      state = state.copyWith(errorMessage: null);
    }
  }
}

// Provider declaration
// StateNotifierProvider<NotifierType, StateType>
final todoProvider = StateNotifierProvider<TodoNotifier, TodoState>((ref) {
  return TodoNotifier();
});
```

**Explanation:**

- **`extends StateNotifier<StateType>`**: The base class for complex state management. It requires an initial state in the constructor (`super(initialState)`).
- **Immutable state**: The state object (TodoState) should never be modified directly. Always create a new instance using `copyWith`.
- **State updates**: Assign to `state` property to update. The setter automatically notifies listeners (no need to call `notifyListeners()` like in ChangeNotifier).
- **Business logic**: All logic (validation, API calls, calculations) lives in the Notifier, not the widget.
- **Error handling**: Use try-catch in async methods and update state with error messages.
- **List operations**: When updating lists, always create new lists (`[...oldList, newItem]` or `.map().toList()`) rather than modifying the existing list.
- **StateNotifierProvider**: Declares the provider. It takes a callback that creates the notifier instance.

### **Using StateNotifierProvider in UI**

```dart
class TodoScreen extends ConsumerWidget {
  final _textController = TextEditingController();
  
  @override
  Widget build(BuildContext context, WidgetRef ref) {
    // Watch the state (TodoState), not the notifier
    final todoState = ref.watch(todoProvider);
    // To access methods, we need the notifier:
    final todoNotifier = ref.read(todoProvider.notifier);
    
    return Scaffold(
      appBar: AppBar(
        title: Text('Todo App (${todoState.pendingTodos.length} pending)'),
        actions: [
          if (todoState.isLoading)
            Padding(
              padding: EdgeInsets.all(16),
              child: CircularProgressIndicator(color: Colors.white),
            ),
        ],
      ),
      body: Column(
        children: [
          // Error display
          if (todoState.errorMessage != null)
            Container(
              color: Colors.red.shade100,
              padding: EdgeInsets.all(16),
              child: Row(
                children: [
                  Icon(Icons.error, color: Colors.red),
                  SizedBox(width: 8),
                  Expanded(child: Text(todoState.errorMessage!)),
                  IconButton(
                    icon: Icon(Icons.close),
                    onPressed: () => todoNotifier.clearError(),
                  ),
                ],
              ),
            ),
          
          // Input section
          Padding(
            padding: EdgeInsets.all(16),
            child: Row(
              children: [
                Expanded(
                  child: TextField(
                    controller: _textController,
                    decoration: InputDecoration(
                      hintText: 'Add a new todo',
                      border: OutlineInputBorder(),
                    ),
                    onSubmitted: (value) {
                      if (value.isNotEmpty) {
                        todoNotifier.addTodo(value);
                        _textController.clear();
                      }
                    },
                  ),
                ),
                SizedBox(width: 8),
                ElevatedButton(
                  onPressed: () {
                    if (_textController.text.isNotEmpty) {
                      todoNotifier.addTodo(_textController.text);
                      _textController.clear();
                    }
                  },
                  child: Text('Add'),
                ),
              ],
            ),
          ),
          
          // Stats
          Padding(
            padding: EdgeInsets.symmetric(horizontal: 16),
            child: Row(
              mainAxisAlignment: MainAxisAlignment.spaceAround,
              children: [
                Text('Total: ${todoState.todoCount}'),
                Text('Done: ${todoState.completedCount}'),
                Text('Pending: ${todoState.pendingTodos.length}'),
              ],
            ),
          ),
          
          // Todo list
          Expanded(
            child: todoState.todos.isEmpty
                ? Center(
                    child: Column(
                      mainAxisAlignment: MainAxisAlignment.center,
                      children: [
                        Icon(Icons.inbox, size: 64, color: Colors.grey),
                        Text('No todos yet'),
                        ElevatedButton(
                          onPressed: () => todoNotifier.loadTodos(),
                          child: Text('Load Sample Data'),
                        ),
                      ],
                    ),
                  )
                : ListView.builder(
                    itemCount: todoState.todos.length,
                    itemBuilder: (context, index) {
                      final todo = todoState.todos[index];
                      return ListTile(
                        leading: Checkbox(
                          value: todo.isCompleted,
                          onChanged: (value) => todoNotifier.toggleTodo(todo.id),
                        ),
                        title: Text(
                          todo.title,
                          style: TextStyle(
                            decoration: todo.isCompleted
                                ? TextDecoration.lineThrough
                                : null,
                            color: todo.isCompleted ? Colors.grey : null,
                          ),
                        ),
                        subtitle: Text(
                          'Created: ${todo.createdAt.toString().substring(0, 16)}',
                        ),
                        trailing: IconButton(
                          icon: Icon(Icons.delete, color: Colors.red),
                          onPressed: () => todoNotifier.removeTodo(todo.id),
                        ),
                      );
                    },
                  ),
          ),
        ],
      ),
    );
  }
}
```

**Explanation:**

- **`ref.watch(todoProvider)`**: Returns the current `TodoState`. The widget rebuilds when state changes.
- **`ref.read(todoProvider.notifier)`**: Returns the `TodoNotifier` instance (only once, doesn't listen to changes). Use this to call methods.
- **Separation of concerns**: UI only displays state and calls methods. Logic lives in the Notifier.
- **Computed properties**: Use the state's computed properties (`pendingTodos`, `completedCount`) directly in the UI rather than calculating in the widget.
- **Loading states**: Check `todoState.isLoading` to show loading indicators.
- **Error handling**: Check `todoState.errorMessage` and provide a way to clear errors.

---

## **13.4 AutoDispose and Family Modifiers**

Modifiers change the behavior of providers. The two most important are `autoDispose` (cleans up when not used) and `family` (creates parameterized providers).

### **AutoDispose Modifier**

```dart
import 'package:flutter_riverpod/flutter_riverpod.dart';

// Without autoDispose: Provider lives forever (until app closes)
final permanentProvider = Provider<String>((ref) {
  print('Permanent provider created');
  return 'I live forever';
});

// With autoDispose: Provider is destroyed when no longer watched
final autoDisposeProvider = Provider.autoDispose<String>((ref) {
  print('AutoDispose provider created');
  
  // onDispose callback runs when provider is destroyed
  ref.onDispose(() {
    print('AutoDispose provider disposed!');
  });
  
  return 'I clean up after myself';
});

// Practical example: Caching network requests
final userProvider = FutureProvider.autoDispose.family<User, String>((ref, userId) async {
  // Cancel the request if the user leaves the screen before it completes
  final cancelToken = CancelToken();
  
  ref.onDispose(() {
    // Cancel the HTTP request when provider is disposed
    cancelToken.cancel();
    print('Request cancelled for user $userId');
  });
  
  final response = await dio.get(
    'https://api.example.com/users/$userId',
    cancelToken: cancelToken,
  );
  
  return User.fromJson(response.data);
});

// StateNotifier with autoDispose
final searchProvider = StateNotifierProvider.autoDispose<SearchNotifier, SearchState>((ref) {
  final notifier = SearchNotifier();
  
  // Clean up resources when disposed
  ref.onDispose(() {
    notifier.dispose(); // Close streams, timers, etc.
  });
  
  return notifier;
});

class SearchNotifier extends StateNotifier<SearchState> {
  SearchNotifier() : super(SearchState());
  
  Timer? _debounceTimer;
  
  void search(String query) {
    // Cancel previous timer
    _debounceTimer?.cancel();
    
    // Debounce: wait 500ms after user stops typing
    _debounceTimer = Timer(Duration(milliseconds: 500), () {
      // Perform search...
    });
  }
  
  @override
  void dispose() {
    _debounceTimer?.cancel();
    super.dispose();
  }
}
```

**Explanation:**

- **`.autoDispose`**: Appended to any provider type. When all widgets stop watching the provider, it automatically disposes itself, freeing memory.
- **When to use**: Use for screen-specific data, search queries, form state, or any data that shouldn't persist after leaving the screen.
- **`ref.onDispose()`**: Register a callback that runs when the provider is destroyed. Use this to cancel HTTP requests, close streams, stop timers, etc.
- **HTTP cancellation**: In the example, if the user navigates away before the API call completes, the request is cancelled, preventing unnecessary network usage and state updates on disposed widgets.
- **Debounce pattern**: AutoDispose is perfect for search fields where you want to cancel pending operations when the user leaves the screen.

### **Family Modifier**

```dart
// Family creates parameterized providers
// Each unique parameter gets its own provider instance

// A provider that takes a user ID and returns user data
final userProvider = FutureProvider.family<User, String>((ref, userId) async {
  // userId is the parameter passed when watching the provider
  final response = await http.get(
    Uri.parse('https://api.example.com/users/$userId'),
  );
  return User.fromJson(jsonDecode(response.body));
});

// StateNotifier with family - each ID gets its own state
final itemEditProvider = StateNotifierProvider.family<ItemEditNotifier, ItemState, String>(
  (ref, itemId) {
    // Create a notifier specific to this itemId
    return ItemEditNotifier(itemId: itemId);
  },
);

class ItemEditNotifier extends StateNotifier<ItemState> {
  final String itemId;
  
  ItemEditNotifier({required this.itemId}) : super(ItemState()) {
    // Load item data when created
    loadItem();
  }
  
  Future<void> loadItem() async {
    // Load specific item by itemId
  }
}

// Combining autoDispose and family
final messageProvider = StreamProvider.autoDispose.family<List<Message>, String>(
  (ref, chatRoomId) {
    // Listen to messages for a specific chat room
    // Automatically disposes when leaving the chat room
    return FirebaseFirestore.instance
        .collection('chatRooms')
        .doc(chatRoomId)
        .collection('messages')
        .orderBy('timestamp')
        .snapshots()
        .map((snapshot) => snapshot.docs.map((doc) => Message.fromDoc(doc)).toList());
  },
);

// Usage in widgets
class UserProfileScreen extends ConsumerWidget {
  final String userId;
  
  const UserProfileScreen({Key? key, required this.userId}) : super(key: key);
  
  @override
  Widget build(BuildContext context, WidgetRef ref) {
    // Watch the specific provider instance for this userId
    final userAsync = ref.watch(userProvider(userId));
    
    return Scaffold(
      appBar: AppBar(title: Text('User Profile')),
      body: userAsync.when(
        data: (user) => UserDetails(user: user),
        loading: () => Center(child: CircularProgressIndicator()),
        error: (err, stack) => Center(child: Text('Error: $err')),
      ),
    );
  }
}

// Dynamic family parameters
class ProductListScreen extends ConsumerWidget {
  @override
  Widget build(BuildContext context, WidgetRef ref) {
    final searchQuery = ref.watch(searchQueryProvider);
    
    // The provider instance changes when searchQuery changes
    // Old instance is disposed, new one created
    final productsAsync = ref.watch(searchProductsProvider(searchQuery));
    
    return Scaffold(
      body: productsAsync.when(
        data: (products) => ListView.builder(
          itemCount: products.length,
          itemBuilder: (context, index) => ProductTile(product: products[index]),
        ),
        loading: () => Center(child: CircularProgressIndicator()),
        error: (err, stack) => Center(child: Text('Error')),
      ),
    );
  }
}

final searchProductsProvider = FutureProvider.autoDispose.family<List<Product>, String>(
  (ref, query) async {
    // Auto-dispose ensures we don't keep old search results in memory
    final response = await http.get(
      Uri.parse('https://api.example.com/products?search=$query'),
    );
    return parseProducts(response.body);
  },
);
```

**Explanation:**

- **`.family<ReturnType, ParameterType>`**: Creates a provider that accepts a parameter. Each unique parameter value gets its own cached instance.
- **Parameters**: Must have proper `==` and `hashCode` implementations (primitives, Strings, or classes that implement value equality).
- **Usage**: `ref.watch(providerName(parameter))` - pass the parameter when watching.
- **Auto-cleanup**: When combined with `autoDispose`, changing the parameter disposes the old instance and creates a new one.
- **Use cases**: User profiles by ID, product details by ID, chat rooms by room ID, search results by query string.
- **Memory management**: Family providers cache instances by parameter. If you watch `userProvider('123')` in multiple places, you get the same instance.

---

## **13.5 AsyncValue and Error Handling**

`AsyncValue` is a powerful type that represents the state of an asynchronous operation (loading, error, or data). It eliminates null checks and error handling boilerplate.

### **Understanding AsyncValue**

```dart
import 'package:flutter_riverpod/flutter_riverpod.dart';

// AsyncValue has three states:
// 1. AsyncLoading() - Operation in progress
// 2. AsyncError(error, stackTrace) - Operation failed
// 3. AsyncData(value) - Operation succeeded

// FutureProvider automatically returns AsyncValue<T>
final postsProvider = FutureProvider<List<Post>>((ref) async {
  final response = await http.get(Uri.parse('https://api.example.com/posts'));
  if (response.statusCode == 200) {
    return parsePosts(response.body);
  } else {
    throw Exception('Failed to load posts');
  }
});

// StreamProvider also returns AsyncValue<T>
final clockProvider = StreamProvider<DateTime>((ref) {
  return Stream.periodic(Duration(seconds: 1), (_) => DateTime.now());
});

class AsyncValueDemo extends ConsumerWidget {
  @override
  Widget build(BuildContext context, WidgetRef ref) {
    final postsAsync = ref.watch(postsProvider);
    
    // Method 1: Using .when() - most common and readable
    return postsAsync.when(
      data: (posts) {
        // Build UI with data
        return ListView.builder(
          itemCount: posts.length,
          itemBuilder: (context, index) => ListTile(
            title: Text(posts[index].title),
          ),
        );
      },
      loading: () {
        // Show loading indicator
        return Center(child: CircularProgressIndicator());
      },
      error: (error, stackTrace) {
        // Show error message
        return Center(
          child: Column(
            mainAxisAlignment: MainAxisAlignment.center,
            children: [
              Icon(Icons.error, color: Colors.red, size: 48),
              Text('Error: $error'),
              ElevatedButton(
                onPressed: () {
                  // Refresh the provider (retry the future)
                  ref.refresh(postsProvider);
                },
                child: Text('Retry'),
              ),
            ],
          ),
        );
      },
    );
    
    // Method 2: Using pattern matching (Dart 3.0+)
    // return switch (postsAsync) {
    //   AsyncData(:final value) => ListView.builder(
    //       itemCount: value.length,
    //       itemBuilder: (context, index) => Text(value[index].title),
    //     ),
    //   AsyncError(:final error) => Text('Error: $error'),
    //   _ => CircularProgressIndicator(),
    // };
    
    // Method 3: Manual checking (not recommended, but possible)
    // if (postsAsync is AsyncLoading) {
    //   return CircularProgressIndicator();
    // } else if (postsAsync is AsyncError) {
    //   return Text('Error');
    // } else if (postsAsync is AsyncData) {
    //   return Text('${postsAsync.value}');
    // }
  }
}
```

**Explanation:**

- **`AsyncValue<T>`**: An abstract class with three concrete implementations: `AsyncData`, `AsyncLoading`, and `AsyncError`.
- **`.when()`**: The most ergonomic way to handle all three states. It's exhaustive (you must handle all cases).
- **Type safety**: `data` callback receives the actual typed value (List<Post>), not dynamic.
- **Error handling**: `error` callback receives both the error object and stack trace.
- **Refreshing**: `ref.refresh(provider)` re-executes the provider's create function, effectively retrying the operation.
- **Pattern matching**: Dart 3.0+ allows using switch expressions with AsyncValue for even cleaner syntax.

### **Combining AsyncValues**

```dart
// When you need data from multiple async providers
final userProvider = FutureProvider<User>((ref) async {
  return fetchUser();
});

final postsProvider = FutureProvider<List<Post>>((ref) async {
  return fetchPosts();
});

// Provider that depends on both
final userProfileProvider = FutureProvider<UserProfile>((ref) async {
  // Wait for both futures to complete
  final user = await ref.watch(userProvider.future);
  final posts = await ref.watch(postsProvider.future);
  
  return UserProfile(user: user, posts: posts);
});

// Or using Future.wait for parallel execution
final dashboardProvider = FutureProvider<Dashboard>((ref) async {
  // Execute multiple futures in parallel
  final results = await Future.wait([
    ref.watch(userProvider.future),
    ref.watch(postsProvider.future),
    ref.watch(commentsProvider.future),
  ]);
  
  return Dashboard(
    user: results[0] as User,
    posts: results[1] as List<Post>,
    comments: results[2] as List<Comment>,
  );
});

// Handling errors in combined providers
class DashboardScreen extends ConsumerWidget {
  @override
  Widget build(BuildContext context, WidgetRef ref) {
    final dashboardAsync = ref.watch(dashboardProvider);
    
    return dashboardAsync.when(
      data: (dashboard) => DashboardView(dashboard: dashboard),
      loading: () => Scaffold(
        body: Center(child: CircularProgressIndicator()),
      ),
      error: (error, stack) {
        // Check error type for specific handling
        if (error is NetworkException) {
          return NetworkErrorView(onRetry: () => ref.refresh(dashboardProvider));
        } else if (error is AuthException) {
          return AuthErrorView();
        }
        return GenericErrorView(error: error);
      },
    );
  }
}
```

**Explanation:**

- **`ref.watch(provider.future)`**: For FutureProvider, `.future` gives you the underlying Future, allowing you to use `await` and `Future.wait`.
- **Parallel execution**: `Future.wait` runs multiple futures concurrently rather than sequentially, improving performance.
- **Error propagation**: If any of the watched providers error, the combined provider also errors.
- **Specific error handling**: Check error types in the error callback to show appropriate UI (network error vs auth error).

---

## **13.6 Testing Riverpod Providers**

One of Riverpod's biggest advantages is testability. You can test providers in isolation without widgets or BuildContext.

### **Unit Testing Providers**

```dart
import 'package:flutter_test/flutter_test.dart';
import 'package:flutter_riverpod/flutter_riverpod.dart';

// Simple provider to test
final greetingProvider = Provider<String>((ref) => 'Hello');

// StateNotifier to test
class Counter extends StateNotifier<int> {
  Counter() : super(0);
  
  void increment() => state++;
  void decrement() => state--;
  void reset() => state = 0;
}

final counterProvider = StateNotifierProvider<Counter, int>((ref) => Counter());

void main() {
  group('Provider Tests', () {
    test('greetingProvider returns correct value', () {
      // Create a ProviderContainer (the heart of Riverpod)
      final container = ProviderContainer();
      
      // Read the provider value
      final greeting = container.read(greetingProvider);
      
      expect(greeting, 'Hello');
      
      // Dispose the container to clean up
      container.dispose();
    });
    
    test('Counter increments correctly', () {
      final container = ProviderContainer();
      
      // Get the notifier
      final counter = container.read(counterProvider.notifier);
      
      // Initial state
      expect(container.read(counterProvider), 0);
      
      // Act
      counter.increment();
      
      // Assert
      expect(container.read(counterProvider), 1);
      
      counter.increment();
      expect(container.read(counterProvider), 2);
      
      counter.decrement();
      expect(container.read(counterProvider), 1);
      
      counter.reset();
      expect(container.read(counterProvider), 0);
      
      container.dispose();
    });
    
    test('Provider overrides', () {
      // Override a provider with a mock value
      final container = ProviderContainer(
        overrides: [
          // Replace the real implementation with a mock
          greetingProvider.overrideWithValue('Goodbye'),
        ],
      );
      
      final greeting = container.read(greetingProvider);
      expect(greeting, 'Goodbye');
      
      container.dispose();
    });
  });
  
  group('Widget Tests with Riverpod', () {
    testWidgets('Counter increments in UI', (tester) async {
      await tester.pumpWidget(
        ProviderScope(
          child: MaterialApp(
            home: Consumer(
              builder: (context, ref, child) {
                final count = ref.watch(counterProvider);
                final notifier = ref.read(counterProvider.notifier);
                
                return Scaffold(
                  body: Text('$count'),
                  floatingActionButton: FloatingActionButton(
                    onPressed: () => notifier.increment(),
                    child: Icon(Icons.add),
                  ),
                );
              },
            ),
          ),
        ),
      );
      
      // Initial state
      expect(find.text('0'), findsOneWidget);
      
      // Tap button
      await tester.tap(find.byType(FloatingActionButton));
      await tester.pump(); // Rebuild
      
      // Verify increment
      expect(find.text('1'), findsOneWidget);
    });
    
    testWidgets('Provider overrides in widget tests', (tester) async {
      // Create a mock notifier
      final mockCounter = MockCounter();
      
      await tester.pumpWidget(
        ProviderScope(
          overrides: [
            // Override with mock
            counterProvider.overrideWith((ref) => mockCounter),
          ],
          child: MaterialApp(home: TestWidget()),
        ),
      );
      
      // Test with mock...
    });
  });
}

class MockCounter extends StateNotifier<int> implements Counter {
  MockCounter() : super(0);
  
  @override
  void increment() {
    // Mock implementation
    state = 999; // Mock value for testing
  }
  
  @override
  void decrement() {}
  
  @override
  void reset() {}
}

class TestWidget extends ConsumerWidget {
  @override
  Widget build(BuildContext context, WidgetRef ref) {
    final count = ref.watch(counterProvider);
    return Text('$count');
  }
}
```

**Explanation:**

- **`ProviderContainer`**: The core class that holds provider states. In tests, you create this directly instead of using ProviderScope.
- **`container.read(provider)`**: Reads a provider's value without listening. Use this for testing logic.
- **`container.read(provider.notifier)`**: Gets the notifier to call methods.
- **Overrides**: Use `ProviderContainer(overrides: [...])` to replace providers with mocks or test values. This is much cleaner than Provider's dependency injection.
- **Widget testing**: Wrap widgets in `ProviderScope`. You can provide overrides at this level too.
- **No BuildContext needed**: Unlike Provider, you don't need to build widgets to test business logic.

---

## **Chapter Summary**

In this chapter, we covered Riverpod, the next-generation state management solution:

### **Key Takeaways:**

1. **Riverpod vs Provider**: Riverpod is compile-safe, doesn't require BuildContext, auto-disposes by default, and is more testable.
2. **ProviderScope**: The root widget that must wrap your app to enable Riverpod.
3. **Provider Types**:
   - `Provider`: Immutable, read-only values
   - `StateProvider`: Simple mutable state (primitives)
   - `StateNotifierProvider`: Complex business logic with immutable state
   - `FutureProvider`: Async operations with automatic loading/error states
   - `StreamProvider`: Real-time data streams
4. **StateNotifier**: Class that holds immutable state and exposes methods to modify it. State updates by assigning to `state` property.
5. **Modifiers**:
   - `.autoDispose`: Automatically dispose when not used (prevents memory leaks)
   - `.family`: Create parameterized providers (e.g., by ID or query string)
   - Can be combined: `.autoDispose.family`
6. **AsyncValue**: Handles loading, error, and data states elegantly with `.when()` method.
7. **Testing**: Use `ProviderContainer` to test providers in isolation without widgets or BuildContext. Use `overrides` to inject mocks.
8. **Best Practices**:
   - Keep providers global and pure
   - Use StateNotifier for complex logic
   - Use autoDispose for screen-specific data
   - Prefer immutable state with copyWith methods

### **Migration from Provider**:
- Replace `ChangeNotifier` with `StateNotifier`
- Replace `ChangeNotifierProvider` with `StateNotifierProvider`
- Remove `Consumer` widgets, use `ConsumerWidget` or `ref.watch` instead
- Move providers from widget tree to global variables
- Add `ProviderScope` at app root

---

**End of Chapter 13**

---

# **Next Chapter: Chapter 14 - BLoC Pattern (Business Logic Component)**

Chapter 14 will explore the BLoC (Business Logic Component) pattern, one of the most popular architectural patterns in Flutter. You'll learn about Events, States, Transitions, and how to implement BLoC using the flutter_bloc library, including Repository pattern integration and HydratedBloc for persistence.